In [1]:
"""
LeNet Architecture

HINTS for layers:

    Convolutional layers:

    tf.nn.conv2d
    tf.nn.max_pool

    For preparing the convolutional layer output for the
    fully connected layers.

    tf.contrib.flatten
"""

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.layers import flatten


# NOTE: Feel free to change these.
EPOCHS = 10
BATCH_SIZE = 64

In [7]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

def conv_layer(input,              # The previous layer.
                   num_input_channels, # Num. channels in prev. layer.
                   filter_size,        # Width and height of each filter.
                   num_filters,        # Number of filters.
                   stride,             # stride size
                   padding):           # padding 'VALID' or 'SAME'
    
    shape = [filter_size, filter_size, num_input_channels, num_filters]
    weights = new_weights(shape=shape)
    biases = new_biases(length=num_filters)
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, stride, stride, 1],
                         padding=padding)
    layer += biases
    return layer

def fully_connected_layer(input,              # The previous layer.
                   num_inputs,                # the number of inputs
                   num_outputs):              # the number of outputs
    
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)
    layer = tf.matmul(input, weights) + biases
    return layer

Convolution layer 1. The output shape should be 28x28x6.

Activation 1. Your choice of activation function.

Pooling layer 1. The output shape should be 14x14x6.

Convolution layer 2. The output shape should be 10x10x16.

Activation 2. Your choice of activation function.

Pooling layer 2. The output shape should be 5x5x16.

Flatten layer. Flatten the output shape of the final pooling layer such that it's 1D instead of 3D. The easiest way to do is by using tf.contrib.layers.flatten, which is already imported for you.

Fully connected layer 1. This should have 120 outputs.

Activation 3. Your choice of activation function.

Fully connected layer 2. This should have 10 outputs.

In [10]:
# LeNet architecture:
# INPUT -> CONV -> ACT -> POOL -> CONV -> ACT -> POOL -> FLATTEN -> FC -> ACT -> FC
#
# Don't worry about anything else in the file too much, all you have to do is
# create the LeNet and return the result of the last fully connected layer.
def LeNet(x):
    # Reshape from 2D to 4D. This prepares the data for
    # convolutional and pooling layers.
    x = tf.reshape(x, (-1, 28, 28, 1))
    # Squish values from 0-255 to 0-1.
    x /= 255.
    # Resize to 32x32.
    x = tf.image.resize_images(x, (32, 32))
    # Convolution layer 1. The output shape should be 28x28x6.
    x=conv_layer(input=x, num_input_channels=1, filter_size=5, num_filters=6, stride=1, padding='VALID')
    # Activation 1. Your choice of activation function.
    x=tf.nn.relu(x)
    # Pooling layer 1. The output shape should be 14x14x6.
    x=tf.nn.max_pool(value=x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    # Convolution layer 2. The output shape should be 10x10x16.
    x=conv_layer(input=x, num_input_channels=6, filter_size=5, num_filters=16, stride=1, padding='VALID')
    # Activation 2. Your choice of activation function.
    x=tf.nn.relu(x)
    # Pooling layer 2. The output shape should be 5x5x16.
    x=tf.nn.max_pool(value=x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    # Flatten layer. Flatten the output shape of the final pooling layer such that it's 1D instead of 3D. The easiest way to do is by using tf.contrib.layers.flatten, which is already imported for you.
    x=tf.contrib.layers.flatten(x)
    # Fully connected layer 1. This should have 120 outputs.
    x=fully_connected_layer(x, 400, 120)
    # Activation 3. Your choice of activation function.
    x=tf.nn.relu(x)
    # Fully connected layer 2. This should have 10 outputs.
    x=fully_connected_layer(x, 120, 10)
    # Return the result of the last fully connected layer.
    return x


In [11]:
# MNIST consists of 28x28x1, grayscale images.
x = tf.placeholder(tf.float32, (None, 784))
# Classify over 10 digits 0-9.
y = tf.placeholder(tf.float32, (None, 10))
# Create the LeNet.
fc2 = LeNet(x)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(fc2, y))
opt = tf.train.AdamOptimizer()
train_op = opt.minimize(loss_op)
correct_prediction = tf.equal(tf.argmax(fc2, 1), tf.argmax(y, 1))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


def eval_data(dataset):
    """
    Given a dataset as input returns the loss and accuracy.
    """
    steps_per_epoch = dataset.num_examples // BATCH_SIZE
    num_examples = steps_per_epoch * BATCH_SIZE
    total_acc, total_loss = 0, 0
    for step in range(steps_per_epoch):
        batch_x, batch_y = dataset.next_batch(BATCH_SIZE)
        loss, acc = sess.run([loss_op, accuracy_op], feed_dict={x: batch_x, y: batch_y})
        total_acc += (acc * batch_x.shape[0])
        total_loss += (loss * batch_x.shape[0])
    return total_loss/num_examples, total_acc/num_examples


if __name__ == '__main__':
    # Load data
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        steps_per_epoch = mnist.train.num_examples // BATCH_SIZE
        num_examples = steps_per_epoch * BATCH_SIZE

        # Train model
        for i in range(EPOCHS):
            for step in range(steps_per_epoch):
                batch_x, batch_y = mnist.train.next_batch(BATCH_SIZE)
                loss = sess.run(train_op, feed_dict={x: batch_x, y: batch_y})

            val_loss, val_acc = eval_data(mnist.validation)
            print("EPOCH {} ...".format(i+1))
            print("Validation loss = {}".format(val_loss))
            print("Validation accuracy = {}".format(val_acc))

        # Evaluate on the test data
        test_loss, test_acc = eval_data(mnist.test)
        print("Test loss = {}".format(test_loss))
        print("Test accuracy = {}".format(test_acc))

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
EPOCH 1 ...
Validation loss = 0.5514993079197712
Validation accuracy = 0.8303285256410257
EPOCH 2 ...
Validation loss = 0.37542294672666454
Validation accuracy = 0.8888221153846154
EPOCH 3 ...
Validation loss = 0.30777175839130694
Validation accuracy = 0.9104567307692307
EPOCH 4 ...
Validation loss = 0.27229593818386394
Validation accuracy = 0.9198717948717948
EPOCH 5 ...
Validation loss = 0.24423356774525765
Validation accuracy = 0.9268830128205128
EPOCH 6 ...
Validation loss = 0.2304763797766123
Validation accuracy = 0.9320913461538461
EPOCH